#**Decision Trees**

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Define the ID3 decision tree class

In [2]:
class DecisionTree:
    def __init__(self, maxdepth=np.inf):
        self.tree = {}
        self.maxdepth = maxdepth

    # Calculate the entropy of a given dataset, the distribution is over the target class.
    def calculate_entropy(self, data):
        labels = data.iloc[:, -1]
        labels_distribution = np.sum(labels == 'unacc'), np.sum(labels == 'acc'), np.sum(labels == 'good'), np.sum(
            labels == 'vgood')
        labels_distribution = np.array(labels_distribution) / len(labels)
        labels_entropy = -np.sum(labels_distribution * np.log(labels_distribution + 1e-10))
        return labels_entropy

    # Calculate the information gain of a feature based on its value
    def calculate_information_gain(self, data, feature):
        total_entropy = self.calculate_entropy(data)
        information_gain = total_entropy

        distincts = list(set(data[feature]))  # get the values of the feature

        feature_entropy_distribution = np.array(
            [np.sum(data[feature] == distinct) * self.calculate_entropy(self.filter_data(data, feature, distinct)) for
             distinct in distincts])
        feature_entropy_distribution /= data.shape[0]
        information_gain -= np.sum(feature_entropy_distribution)
        return information_gain

    def filter_data(self, data, feature, value):
        return data[data[feature] == value].drop(feature, axis=1)

    def create_tree(self, data, depth=0):
        # Recursive function to create the decision tree
        labels = data.iloc[:, -1]

        # Base case: if all labels are the same, return the label
        if len(np.unique(labels)) == 1:
            return list(labels)[0]

        features = data.columns.tolist()[:-1]

        # Base case: if there are no features left to split on, return the majority label
        if len(features) == 0 or depth == self.maxdepth:
            unique_labels, label_counts = np.unique(labels, return_counts=True)
            majority_label = unique_labels[label_counts.argmax()]
            return majority_label

        selected_feature = None
        best_gain = 0

        # Select feature that maximizes gain
        for feature in features:
            gain = self.calculate_information_gain(data, feature)
            if gain > best_gain:
                selected_feature = feature
                best_gain = gain

        # Create the tree node
        tree_node = {}

        distincts = list(set(data[selected_feature]))
        for value in distincts:
            new_data = self.filter_data(data, selected_feature, value)
            tree_node[(selected_feature, value)] = self.create_tree(new_data, depth + 1)

        return tree_node

    def fit(self, data):
        self.tree = self.create_tree(data)

    def predict(self, X):
        X = [row[1] for row in X.iterrows()]

        # Predict the labels for new data points
        predictions = []

        for row in X:
            current_node = self.tree
            while isinstance(current_node, dict):
                split_condition = next(iter(current_node))
                feature, value = split_condition
                current_node = current_node[feature, row[feature]]
            predictions.append(current_node)

        return predictions

    def _plot(self, tree, indent):
        depth = 1
        for key, value in tree.items():
            if isinstance(value, dict):
                print(" " * indent + str(key) + ":")
                depth = max(depth, 1 + self._plot(value, indent + 2))
            else:
                print(" " * indent + str(key) + ": " + str(value))
        return depth

    def plot(self):
        depth = self._plot(self.tree, 0)
        print(f'depth is {depth}')

In [3]:
data = pd.read_csv('cars.csv')

## SECTION A

In [4]:
tree = DecisionTree()
tree.fit(data)
tree.plot()

('safety', 'low'): unacc
('safety', 'high'):
  ('persons', '2'): unacc
  ('persons', 'more'):
    ('buying', 'low'):
      ('maint', 'low'):
        ('lug_boot', 'big'): vgood
        ('lug_boot', 'small'):
          ('doors', '2'): unacc
          ('doors', '3'): good
          ('doors', '5more'): good
          ('doors', '4'): good
        ('lug_boot', 'med'):
          ('doors', '2'): good
          ('doors', '3'): vgood
          ('doors', '5more'): vgood
          ('doors', '4'): vgood
      ('maint', 'high'):
        ('lug_boot', 'big'): vgood
        ('lug_boot', 'small'):
          ('doors', '2'): unacc
          ('doors', '3'): acc
          ('doors', '5more'): acc
          ('doors', '4'): acc
        ('lug_boot', 'med'):
          ('doors', '2'): acc
          ('doors', '3'): vgood
          ('doors', '5more'): vgood
          ('doors', '4'): vgood
      ('maint', 'med'):
        ('lug_boot', 'big'): vgood
        ('lug_boot', 'small'):
          ('doors', '2'): unacc
      

## SECTION B

In [5]:
train, test = train_test_split(data, test_size=0.2, random_state=7)

Get training accuracy:

In [6]:
tree = DecisionTree()
tree.fit(train)
pred = tree.predict(train.iloc[:, :-1])
acc = (pred == train.iloc[:,-1]).sum() / len(train)
print(f'Training accuracy is {acc}')

Training accuracy is 1.0


## SECTION C

In [7]:
train, test = train_test_split(data, test_size=0.2, random_state=7)
tree = DecisionTree(maxdepth=5)
tree.fit(train)
pred = tree.predict(train.iloc[:, :-1])
acc = (pred == train.iloc[:,-1]).sum() / len(train)
print(f'Training accuracy is {acc}')

pred = tree.predict(test.iloc[:, :-1])
acc = (pred == test.iloc[:,-1]).sum() / len(test)
print(f'Test accuracy is {acc}')

Training accuracy is 0.9688856729377714
Test accuracy is 0.9248554913294798
